In [2]:
!pip install ultralytics
!pip install supervision==0.1.0
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.7/641.7 kB 6.4 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO
import supervision as sv
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from tqdm.notebook import tqdm

In [11]:
import os
HOME = os.getcwd()
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [12]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [9]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [14]:
SOURCE_VIDEO_PATH="/content/drive/MyDrive/Speed Estimation/DJI_0850.mp4"
OUTPUT_VIDEO_PATH="/content/drive/MyDrive/Speed Estimation/output.mp4"
MODEL_PATH="/content/drive/MyDrive/Speed Estimation/best.pt"
video_info= VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
model= YOLO(MODEL_PATH)
CLASS_NAMES_DICT= model.names

In [ ]:
from typing import List
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [ ]:
import math

def rescale_factor(angle=78, height=180, width=1080):
  return ((math.tan(angle/2)*height)*2)/width
rescale_factor()
video_info.width

3840

In [ ]:
def get_center(box):
    x1, y1, x2, y2 = box
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    return center_x, center_y

def calculate_speed(current_box, previous_box, time_interval=0.1):
    current_center = get_center(current_box)
    previous_center = get_center(previous_box)
    rescale_x= rescale_factor(width=video_info.width)
    rescale_y= rescale_factor(width=video_info.height)
    displacement = math.sqrt(((current_center[0] - previous_center[0])*rescale_x)**2 + ((current_center[1] - previous_center[1])*rescale_y)**2)
    speed = displacement / time_interval
    return speed


# **Code for Speed Estimation**

In [ ]:
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=1, text_thickness=1, text_scale=.5)
current_position = {}
previous_position={}
labels=[]
data_list= []
frame_number=0
# open target video file
with VideoSink(OUTPUT_VIDEO_PATH,video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
        labels=[]
        results = model(frame)
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        # filtering out detections without trackers
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        current_position={x:y for x,y in zip(detections.tracker_id, detections.xyxy)}

        for _, _, class_id, tracker_id in detections:
          if ((tracker_id in previous_position.keys()) and (tracker_id in current_position.keys())):
            speed=calculate_speed(current_position[tracker_id], previous_position[tracker_id])
          else:
            speed=0
          data={}
          data['frame_number'] = frame_number
          data['tracker_id'] = tracker_id
          data['class']= class_id
          data['speed'] = speed
          # print(data)
          data_list.append(data)
          # print(data_list)
          labels.append(f"#{tracker_id} {speed:0.2f}km/hr")

        # print(previous_position, current_position)
        # updating line counter
        # line_counter.update(detections=detections)
        # annotate and display frame
        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
        # line_annotator.annotate(frame=frame, line_counter=line_counter)
        previous_position=current_position
        # print(labels)
        sink.write_frame(frame)
        frame_number+= 1
        # %matplotlib inline
        # show_frame_in_notebook(frame, (16, 16))
        # break

df=pd.DataFrame(data_list)

  0%|          | 0/9022 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
0: 640x1088 60 Carss, 1 LCV, 8 M2Ws, 2 MAVs, 7 Rickshaws, 1 Truck, 58.6ms
Speed: 8.1ms preprocess, 58.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 61 Carss, 2 LCVs, 10 M2Ws, 2 MAVs, 6 Rickshaws, 1 Truck, 58.4ms
Speed: 7.8ms preprocess, 58.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 64 Carss, 1 LCV, 8 M2Ws, 2 MAVs, 5 Rickshaws, 1 Truck, 58.6ms
Speed: 8.1ms preprocess, 58.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 1 Bus, 59 Carss, 1 LCV, 5 M2Ws, 3 MAVs, 4 Rickshaws, 1 Truck, 60.3ms
Speed: 8.0ms preprocess, 60.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 59 Carss, 1 LCV, 8 M2Ws, 3 MAVs, 3 Rickshaws, 1 Truck, 58.8ms
Speed: 7.5ms preprocess, 58.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 57 Carss, 1 LCV, 6 M2Ws, 2 MAVs, 6 Rickshaws, 1 Truck, 58.6ms
S

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import *

In [ ]:
def calculate_acceleration(df):
    df['acceleration'] = df.groupby('tracker_id')['speed'].diff() / 10  # Assuming 10 frame = 1 second
    return df

In [ ]:
df.head()

,frame_number,tracker_id,class,speed,acceleration
0,0,1,1,0.0,NaN
1,0,2,1,0.0,NaN
2,0,3,6,0.0,NaN
3,0,4,1,0.0,NaN
4,0,5,1,0.0,NaN


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
df = calculate_acceleration(df)
#df = calculate_tangential_acceleration(df)

def plot_graphs(change):
    tracker_id = change['new']
    specific_df = df[df['tracker_id'] == tracker_id]
    print('\n')
    print('Class : ',specific_df['class'][0])
    print('\n')

    plt.figure()
    plt.plot(specific_df['frame_number'], specific_df['speed'],color='r')
    plt.title('Speed vs Time')
    plt.xlabel('Frame Number')
    plt.ylabel('Speed (km/hr)')
    plt.grid()

    plt.figure()
    plt.plot(specific_df['frame_number'], specific_df['acceleration'],color='g')
    plt.title('Acceleration vs Time')
    plt.xlabel('Frame Number')
    plt.ylabel('Acceleration (km/hr^2)')
    plt.grid()

    # plt.figure()
    # plt.plot(specific_df['frame_number'], specific_df['tangential_acceleration'],color='orange')
    # plt.title('Tangential Acceleration vs Time')
    # plt.xlabel('Frame Number')
    # plt.ylabel('Tangential Acceleration (km/hr^2)')

    # plt.show()

# Create a dropdown for tracker IDs
tracker_ids = df['tracker_id'].unique().tolist()
tracker_id_dropdown = widgets.Dropdown(
    options=tracker_ids,
    value=tracker_ids[0],
    description='Tracker ID:',
    disabled=False,
)

tracker_id_dropdown.observe(plot_graphs, names='value')

# Save DataFrame to CSV
df.to_csv('/content/drive/MyDrive/Speed Estimation/data.csv', index=False, header=True)

# Display the dropdown
display(tracker_id_dropdown)
